In [19]:
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
import cv2

In [18]:
path = r"/Users/wendy/Documents/GitHub/UMAP/Crypto_punks"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
punks = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.png'):
          # adds only the image files to the flowers list
            punks.append(file.name)

In [29]:
data = {}
p = r"/Users/wendy/Documents/GitHub/UMAP/Crypto_punks"
def reshape_images(file):
    img = cv2.imread(file)
    img = np.array(img)
    reshaped_img = img.flatten()
    return (reshaped_img)

for punk in punks:
    # try to extract the features and update the dictionary
    #try:
    feat = reshape_images(punk)
    data[punk] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    #except:
        #with open(p,'wb') as file:
            #pickle.dump(data,file)

In [33]:
data['3975.png']

True